In [ ]:
import matplotlib.pyplot as plt
from glob import glob
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib
import json
import re

sns.set_theme(style="darkgrid")
matplotlib.rcParams['figure.figsize'] = (10, 6)

%matplotlib inline

In [ ]:
def load_results_df(key: str) -> pd.DataFrame:
    results_files = glob(f"results_*-{key}.json")
    all_results = []
    for file in results_files:
        with open(file, "r") as f:
            results = json.load(f)
        model = re.search(r"results_(.*?).json", file).group(1)
        for task, values in results.items():
            for layer, value in enumerate(values):
                all_results.append({
                    "task": task,
                    "model": model,
                    "task/model": f"{task}/{model}",
                    "layer": layer,
                    "count": value,
                })
    return pd.DataFrame(all_results)

In [ ]:
base_results = load_results_df("base")
large_results = load_results_df("large")
base_results.head()

In [ ]:
def plot_results(results: pd.DataFrame, title: str = None):
    sns.barplot(results, x="layer", y="count", hue="task")
    plt.title(title)
    plt.figure()

In [ ]:
for model in base_results.model.unique():
    plot_results(base_results[base_results.model == model], title=model)

for model in large_results.model.unique():
    plot_results(large_results[large_results.model == model], title=model)